In [2]:
import os
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline

In [3]:
os.chdir("E:\Insofe\PH.D\PHD_ML")

In [4]:
train = pd.read_csv('Train-1554810061973.csv')


In [5]:
train.loc[train.Hotelid == 'hotel_101']


,Reviewid,Hotelid,userid,Date,reviewtext,Sentiment
0,Review_1,hotel_101,hotel_1608,"Nov 16, 2007",Nice Marriot View of my king bed room,good
1,Review_2,hotel_101,hotel_6939,"Oct 30, 2007","Good hotel, charges for internet access The Ma...",good
2,Review_3,hotel_101,hotel_3976,"Oct 12, 2007",Small but adequate rooms If you have an early ...,good
3,Review_4,hotel_101,hotel_2851,"Aug 31, 2007","Better than average, some noisy rooms I have s...",good
4,Review_5,hotel_101,hotel_7897,"Jul 18, 2007",Ordinary Although it is highly rated in these ...,bad
5,Review_6,hotel_101,hotel_3297,"Jul 13, 2007",Awesome for early AM flight Wow! what a supris...,excellent
6,Review_7,hotel_101,hotel_5463,"Jul 4, 2007",Good Stay Spent two nights there in Jun. The h...,good
7,Review_8,hotel_101,hotel_9766,"Jun 14, 2007",Great Staff We stayed overnight the last week ...,excellent
8,Review_9,hotel_101,hotel_7042,"Jun 13, 2007",fine choice for seatac The lobby is attractive...,good
9,Review_10,hotel_101,hotel_9805,"May 30, 2007","Great value, looking for a nice hotel. not exp...",good


In [6]:
train.Sentiment[train.Sentiment == 'bad'] = 0
train.Sentiment[train.Sentiment == 'good'] = 1
train.Sentiment[train.Sentiment == 'excellent'] = 2

In [7]:
print('Average count of reviews per hotel in train is {0:.0f}.'.format(train.groupby('Hotelid')['reviewtext'].count().mean()))
#print('Average count of phrases per sentence in test is {0:.0f}.'.format(test.groupby('Hotelid')['reviewtext'].count().mean()))

Average count of reviews per hotel in train is 63.


In [8]:
print('Number of hotels in train: {}. Number of reviews in train: {}.'.format(train.shape[0], len(train.Hotelid.unique())))

Number of hotels in train: 6287. Number of reviews in train: 100.


In [9]:
print('Average word length of reviews in train is {0:.0f}.'.format(np.mean(train['reviewtext'].apply(lambda x: len(x.split(" "))))))

Average word length of reviews in train is 168.


In [10]:
test = pd.read_csv('Test-1555730055539.csv')
#sub = pd.read_csv('../input/movie-review-sentiment-analysis-kernels-only/sampleSubmission.csv', sep=",")

In [11]:
#Adding Sentiment column to test datset and joing train and test for preprocessing

test['Sentiment']=-999
test.head()

,Reviewid,Hotelid,userid,Date,reviewtext,Sentiment
0,Review_11001,hotel_101,hotel_2225,"Dec 13, 2008",Just An Average stay This was just an average ...,-999
1,Review_11002,hotel_101,hotel_5079,"Dec 2, 2008",go elsewhere The place is hugely overpriced an...,-999
2,Review_11003,hotel_101,hotel_8440,"Nov 18, 2008",I Won't Go Back I stayed at the hotel 11/14/08...,-999
3,Review_11004,hotel_101,hotel_4592,"Oct 19, 2008",Good weekend stay My wife and I stay here quit...,-999
4,Review_11005,hotel_101,hotel_5901,"Oct 13, 2008","Great airport stay Lovely indoor pool area, lo...",-999


In [12]:
df=pd.concat([train,test],ignore_index=True)
print(df.shape)
df.tail()

(9755, 6)


,Reviewid,Hotelid,userid,Date,reviewtext,Sentiment
9750,Review_14464,hotel_200,hotel_4764,"Aug 18, 2008","Un hotel encantador, muy recomendable showRe...",-999
9751,Review_14465,hotel_200,hotel_6698,"Apr 20, 2008","Buon albergo showReview(15155815, 'full');",-999
9752,Review_14466,hotel_200,hotel_6390,"Apr 12, 2008","Un hotel eccezionale showReview(14980612, 'f...",-999
9753,Review_14467,hotel_200,hotel_7902,"Mar 28, 2008","Ottimo showReview(14599783, 'full');",-999
9754,Review_14468,hotel_200,hotel_10561,"Mar 22, 2008",Un relax en el ritmo trepidante de New York ...,-999


In [13]:
del train,test
import gc
gc.collect()

35

In [14]:
#cleaning review



In [15]:
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()
from string import punctuation
import re

In [16]:
def clean_review(review_col):
    review_corpus=[]
    for i in range(0,len(review_col)):
        review=str(review_col[i])
        review=re.sub('[^a-zA-Z]',' ',review)
        review=[stemmer.stem(w) for w in word_tokenize(str(review).lower())]
        review=[lemma.lemmatize(w) for w in word_tokenize(str(review).lower())]
        review=' '.join(review)
        review_corpus.append(review)
    return review_corpus

In [17]:

import nltk
nltk.download('wordnet')
df['clean_review']=clean_review(df.reviewtext.values)
df.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\prash\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Reviewid,Hotelid,userid,Date,reviewtext,Sentiment,clean_review
0,Review_1,hotel_101,hotel_1608,"Nov 16, 2007",Nice Marriot View of my king bed room,1,"[ 'nice ' , 'marriot ' , 'view ' , 'of ' , 'my..."
1,Review_2,hotel_101,hotel_6939,"Oct 30, 2007","Good hotel, charges for internet access The Ma...",1,"[ 'good ' , 'hotel ' , 'charg ' , 'for ' , 'in..."
2,Review_3,hotel_101,hotel_3976,"Oct 12, 2007",Small but adequate rooms If you have an early ...,1,"[ 'small ' , 'but ' , 'adequ ' , 'room ' , 'if..."
3,Review_4,hotel_101,hotel_2851,"Aug 31, 2007","Better than average, some noisy rooms I have s...",1,"[ 'better ' , 'than ' , 'averag ' , 'some ' , ..."
4,Review_5,hotel_101,hotel_7897,"Jul 18, 2007",Ordinary Although it is highly rated in these ...,0,"[ 'ordinari ' , 'although ' , 'it ' , 'is ' , ..."


In [18]:
#seperating train and test dataset
df_train=df[df.Sentiment!=-999]
df_train.shape
df_test=df[df.Sentiment==-999]
df_test.drop('Sentiment',axis=1,inplace=True)
print(df_test.shape)
df_test.head()
from keras.utils.np_utils import to_categorical



C:\Users\prash\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


(3468, 6)


C:\Users\prash\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [19]:
del df
gc.collect()

0

In [20]:
#Splitting Train dataset into train and 20% validation se
train_text=df_train.clean_review.values
test_text=df_test.clean_review.values
target=df_train.Sentiment.values
y=to_categorical(target)
print(train_text.shape,target.shape,y.shape)

(6287,) (6287,) (6287, 3)


In [21]:
X_train_text,X_val_text,y_train,y_val=train_test_split(train_text,y,test_size=0.2,stratify=y,random_state=123)
print(X_train_text.shape,y_train.shape)
print(X_val_text.shape,y_val.shape)

(5029,) (5029, 3)
(1258,) (1258, 3)


In [22]:
#Finding number of unique words in train set¶

all_words=' '.join(X_train_text)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
num_unique_word

12893

In [23]:
#Finding max length of a review in train se
r_len=[]
for text in X_train_text:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)
    
MAX_REVIEW_LEN=np.max(r_len)
MAX_REVIEW_LEN

4282

In [25]:
#Building Keras LSTM model¶
#max_features = num_unique_word
max_features = 10000
#max_words = MAX_REVIEW_LEN
max_words = 500
batch_size = 64
epochs = 3
num_classes=3

In [26]:
#Tokenize Text
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train_text))
X_train = tokenizer.texts_to_sequences(X_train_text)
X_val = tokenizer.texts_to_sequences(X_val_text)
X_test = tokenizer.texts_to_sequences(test_text)

In [27]:
#sequence padding
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_val = sequence.pad_sequences(X_val, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
print(X_train.shape,X_val.shape,X_test.shape)

(5029, 500) (1258, 500) (3468, 500)


In [173]:
# LSTM Model
from keras.layers import Input, Dense, Dropout, Embedding, LSTM, Flatten,Activation,BatchNormalization,Conv1D,Conv2D,MaxPooling2D
from keras.models import Sequential
from keras.optimizers import Adam
model1=Sequential()
model1.add(Embedding(max_features,200,mask_zero=True))
model1.add(LSTM(64,dropout=0.2, recurrent_dropout=0.2,return_sequences=True))
model1.add(LSTM(32,dropout=0.2, recurrent_dropout=0.3,return_sequences=False))
model1.add(Dense(num_classes,activation='softmax'))
model1.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_40 (Embedding)     (None, None, 200)         2578600   
_________________________________________________________________
lstm_22 (LSTM)               (None, None, 64)          67840     
_________________________________________________________________
lstm_23 (LSTM)               (None, 32)                12416     
_________________________________________________________________
dense_53 (Dense)             (None, 3)                 99        
Total params: 2,658,955
Trainable params: 2,658,955
Non-trainable params: 0
_________________________________________________________________


In [ ]:
%%time
history1=model1.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Train on 5029 samples, validate on 1258 samples
Epoch 1/3


In [ ]:
y_pred1=model1.predict_classes(X_test,verbose=1)

In [ ]:
sub = pd.read_csv('sampleSubmission.csv')

In [ ]:
sub.Sentiment=y_pred1
sub.Sentiment[sub.Sentiment == 0] = 'bad'
sub.Sentiment[sub.Sentiment == 1] = 'good'
sub.Sentiment[sub.Sentiment == 2] = 'excellent'
sub.to_csv('sub1.csv',index=False)
sub.head()

# 2. CNN

In [51]:
kernel_size = 3
filters = 64
pool_size = 3
gru_node = 256

from keras import regularizers
from keras.layers import Input, Dense, Dropout, Embedding, LSTM, Flatten,Activation,BatchNormalization,Conv1D,Conv2D,GlobalMaxPooling1D,MaxPooling2D
from keras.models import Sequential

model2= Sequential()
model2.add(Embedding(max_features,100,input_length=max_words))
model2.add(Dropout(0.15))

model2.add(Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model2.add(GlobalMaxPooling1D())

model2.add(Dense(128,activation='relu'))
model2.add(Dropout(0.25))

model2.add(Dense(num_classes,activation='softmax'))


model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 500, 100)          1000000   
_________________________________________________________________
dropout_13 (Dropout)         (None, 500, 100)          0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 500, 64)           19264     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 64)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               8320      
_________________________________________________________________
dropout_14 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 387       
Total para

In [52]:
%%time
history2=model2.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=6, batch_size=batch_size, verbose=1)

Train on 5029 samples, validate on 1258 samples
Epoch 1/6
5029/5029 [==============================] - 11s 2ms/step - loss: 1.0040 - acc: 0.5065 - val_loss: 0.9551 - val_acc: 0.5111
Epoch 2/6
5029/5029 [==============================] - 12s 2ms/step - loss: 0.8578 - acc: 0.5776 - val_loss: 0.7708 - val_acc: 0.6288
Epoch 3/6
5029/5029 [==============================] - 14s 3ms/step - loss: 0.6813 - acc: 0.6791 - val_loss: 0.6891 - val_acc: 0.6614
Epoch 4/6
5029/5029 [==============================] - 13s 3ms/step - loss: 0.5407 - acc: 0.7658 - val_loss: 0.6576 - val_acc: 0.6828
Epoch 5/6
5029/5029 [==============================] - 13s 3ms/step - loss: 0.4230 - acc: 0.8314 - val_loss: 0.7034 - val_acc: 0.6741
Epoch 6/6
5029/5029 [==============================] - 13s 3ms/step - loss: 0.3108 - acc: 0.8839 - val_loss: 0.7486 - val_acc: 0.6749
Wall time: 1min 15s


In [48]:
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
filepath="weights-simple.hdf5"
checkpointer = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
history = model2.fit([X_train], batch_size=64, y=y_train, verbose=1, 
          shuffle=True, epochs=8, callbacks=[checkpointer])

Epoch 1/8
5029/5029 [==============================] - 11s 2ms/step - loss: 1.0157 - acc: 0.4985
Epoch 2/8
  64/5029 [..............................] - ETA: 10s - loss: 1.0318 - acc: 0.4844

C:\Users\prash\Anaconda3\lib\site-packages\keras\callbacks.py:434: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


5029/5029 [==============================] - 11s 2ms/step - loss: 0.9069 - acc: 0.5484
Epoch 3/8
5029/5029 [==============================] - 12s 2ms/step - loss: 0.7222 - acc: 0.6480
Epoch 4/8
5029/5029 [==============================] - 12s 2ms/step - loss: 0.6108 - acc: 0.7168
Epoch 5/8
5029/5029 [==============================] - 12s 2ms/step - loss: 0.5167 - acc: 0.7773
Epoch 6/8
5029/5029 [==============================] - 12s 2ms/step - loss: 0.4272 - acc: 0.8276
Epoch 7/8
5029/5029 [==============================] - 12s 2ms/step - loss: 0.3379 - acc: 0.8686
Epoch 8/8
5029/5029 [==============================] - 12s 2ms/step - loss: 0.2544 - acc: 0.9077


In [113]:
y_pred2=model2.predict_classes(X_test, verbose=1)

3468/3468 [==============================] - 5s 1ms/step


In [114]:
sub.Sentiment=y_pred2
sub.Sentiment[sub.Sentiment == 0] = 'bad'
sub.Sentiment[sub.Sentiment == 1] = 'good'
sub.Sentiment[sub.Sentiment == 2] = 'excellent'
sub.to_csv('sub2_next_0428.csv',index=False)
sub.head()

C:\Users\prash\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Reviewid,Sentiment
0,Review_11001,good
1,Review_11002,bad
2,Review_11003,bad
3,Review_11004,good
4,Review_11005,good


# CNN +GRU

In [42]:

from keras.layers import GRU,Input, Dense, Dropout, Embedding, LSTM, Flatten,Activation,BatchNormalization,Conv1D,Conv2D,MaxPooling1D,GlobalMaxPooling1D,MaxPooling2D

model3= Sequential()
model3.add(Embedding(max_features,100,input_length=max_words))
model3.add(Conv1D(64,kernel_size=3,padding='same',activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Dropout(0.25))
model3.add(GRU(128,return_sequences=True))
model3.add(Dropout(0.3))
model3.add(Flatten())
model3.add(Dense(128,activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(3,activation='softmax'))
model3.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1427, 100)         1652000   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1427, 64)          19264     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 713, 64)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 713, 64)           0         
_________________________________________________________________
gru_1 (GRU)                  (None, 713, 128)          74112     
_________________________________________________________________
dropout_4 (Dropout)          (None, 713, 128)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 91264)             0         
__________

In [43]:
%%time
history3=model3.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=10, batch_size=batch_size, verbose=1)

Train on 5029 samples, validate on 1258 samples
Epoch 1/10
5029/5029 [==============================] - 152s 30ms/step - loss: 1.0494 - acc: 0.4727 - val_loss: 0.9923 - val_acc: 0.5111
Epoch 2/10
5029/5029 [==============================] - 167s 33ms/step - loss: 0.8895 - acc: 0.5357 - val_loss: 0.7185 - val_acc: 0.6463
Epoch 3/10
5029/5029 [==============================] - 174s 35ms/step - loss: 0.6201 - acc: 0.7041 - val_loss: 0.6730 - val_acc: 0.6709
Epoch 4/10
5029/5029 [==============================] - 175s 35ms/step - loss: 0.3937 - acc: 0.8282 - val_loss: 0.7638 - val_acc: 0.6852
Epoch 5/10
5029/5029 [==============================] - 184s 37ms/step - loss: 0.2513 - acc: 0.8954 - val_loss: 0.9295 - val_acc: 0.6900
Epoch 6/10
1408/5029 [=======>......................] - ETA: 2:09 - loss: 0.1789 - acc: 0.9283

KeyboardInterrupt: 

In [59]:
y_pred3=model3.predict_classes(X_test, verbose=1)

7504/7504 [==============================] - 49s 7ms/step


In [60]:
sub.Sentiment=y_pred3
sub.Sentiment[sub.Sentiment == 0] = 'bad'
sub.Sentiment[sub.Sentiment == 1] = 'good'
sub.Sentiment[sub.Sentiment == 2] = 'excellent'
sub.to_csv('sub3.csv',index=False)
sub.head()

C:\Users\prash\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,userid,Sentiment
0,hotel_2225,good
1,hotel_5079,bad
2,hotel_8440,bad
3,hotel_4592,good
4,hotel_5901,excellent


# 4. Bidirectional GRU

In [61]:
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional, SpatialDropout1D
model4 = Sequential()

model4.add(Embedding(max_features, 100, input_length=max_words))
model4.add(SpatialDropout1D(0.25))
model4.add(Bidirectional(GRU(128)))
model4.add(Dropout(0.5))

model4.add(Dense(3, activation='softmax'))
model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model4.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 1427, 100)         1652000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 1427, 100)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               175872    
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 771       
Total params: 1,828,643
Trainable params: 1,828,643
Non-trainable params: 0
_________________________________________________________________


In [63]:
%%time
history4=model4.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Train on 5029 samples, validate on 1258 samples
Epoch 1/3


KeyboardInterrupt: 

In [64]:
y_pred4=model4.predict_classes(X_test, verbose=1)

7504/7504 [==============================] - 94s 13ms/step


In [65]:
sub.Sentiment=y_pred4
sub.Sentiment[sub.Sentiment == 0] = 'bad'
sub.Sentiment[sub.Sentiment == 1] = 'good'
sub.Sentiment[sub.Sentiment == 2] = 'excellent'
sub.to_csv('sub4.csv',index=False)
sub.head()

C:\Users\prash\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,userid,Sentiment
0,hotel_2225,bad
1,hotel_5079,bad
2,hotel_8440,bad
3,hotel_4592,good
4,hotel_5901,good


# 5. Glove word embedding

In [66]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')
    
def get_embed_mat(EMBEDDING_FILE, max_features,embed_dim):
    # word vectors
    embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE, encoding='utf8'))
    print('Found %s word vectors.' % len(embeddings_index))

    # embedding matrix
    word_index = tokenizer.word_index
    num_words = min(max_features, len(word_index) + 1)
    all_embs = np.stack(embeddings_index.values()) #for random init
    embedding_matrix = np.random.normal(all_embs.mean(), all_embs.std(), 
                                        (num_words, embed_dim))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    max_features = embedding_matrix.shape[0]
    
    return embedding_matrix

In [67]:
# embedding matrix
EMBEDDING_FILE = 'glove.6B.100d.txt'
embed_dim = 100 #word vector dim
embedding_matrix = get_embed_mat(EMBEDDING_FILE,max_features,embed_dim)
print(embedding_matrix.shape)

Found 400000 word vectors.


C:\Users\prash\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if sys.path[0] == '':


(16520, 100)


In [68]:
model5 = Sequential()
model5.add(Embedding(max_features, embed_dim, input_length=X_train.shape[1],weights=[embedding_matrix],trainable=True))
model5.add(SpatialDropout1D(0.25))
model5.add(Bidirectional(GRU(128,return_sequences=True)))
model5.add(Bidirectional(GRU(64,return_sequences=False)))
model5.add(Dropout(0.5))
model5.add(Dense(num_classes, activation='softmax'))
model5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model5.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 1427, 100)         1652000   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 1427, 100)         0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1427, 256)         175872    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 128)               123264    
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 387       
Total params: 1,951,523
Trainable params: 1,951,523
Non-trainable params: 0
_________________________________________________________________


In [69]:
%%time
history5=model5.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=4, batch_size=batch_size, verbose=1)

Train on 5029 samples, validate on 1258 samples
Epoch 1/4
5029/5029 [==============================] - 6176s 1s/step - loss: 1.0582 - acc: 0.4754 - val_loss: 1.0098 - val_acc: 0.5087
Epoch 2/4
5029/5029 [==============================] - 7078s 1s/step - loss: 1.0179 - acc: 0.4925 - val_loss: 0.9813 - val_acc: 0.5286
Epoch 3/4
5029/5029 [==============================] - 14129s 3s/step - loss: 0.9876 - acc: 0.5126 - val_loss: 0.9457 - val_acc: 0.5382
Epoch 4/4
5029/5029 [==============================] - 10773s 2s/step - loss: 0.8996 - acc: 0.5546 - val_loss: 0.7819 - val_acc: 0.6161
Wall time: 10h 35min 57s


In [71]:
y_pred5=model5.predict_classes(X_test, verbose=1)

7504/7504 [==============================] - 397s 53ms/step


In [70]:
sub.Sentiment=y_pred5
sub.Sentiment[sub.Sentiment == 0] = 'bad'
sub.Sentiment[sub.Sentiment == 1] = 'good'
sub.Sentiment[sub.Sentiment == 2] = 'excellent'
sub.to_csv('sub5.csv',index=False)
sub.head()

NameError: name 'y_pred5' is not defined

# combine all output

In [ ]:
sub_all=pd.DataFrame({'model1':y_pred1,'model2':y_pred2,'model3':y_pred3,'model4':y_pred4,'model5':y_pred5})
pred_mode=sub_all.agg('mode',axis=1)[0].values
sub_all.head()

In [ ]:
pred_mode=[int(i) for i in pred_mode]

In [ ]:
sub.Sentiment=pred_mode
sub.to_csv('sub_mode.csv',index=False)
sub.head()